In [1149]:
import numpy as np
import matplotlib.pyplot as plt

In [1150]:
def linear_interpolation(x0, y0, x1, y1, x):
    """
    Perform linear interpolation to find the y value corresponding to x.

    Parameters:
    x0, y0: coordinates of the first point
    x1, y1: coordinates of the second point
    x: the x value to interpolate

    Returns:
    y: the interpolated y value
    """
    y = y0 + (y1 - y0) * (x - x0) / (x1 - x0)
    return y

# Example usage:
# x0, y0 = 0, 0
# x1, y1 = 10, 10
# x = 5
# y = linear_interpolation(x0, y0, x1, y1, x)
# print(f"The interpolated value at x={x} is y={y}")

## Given Constants

In [1151]:
gear = "worm"


n_w = None

n_g = 30

N_g = 40 #None # between 17 and 20 - only for spur?, idk abt helical
N_w = 1 #None

D_g = None
D_w = 1.25

P_d = 10#None #pitch diameter
p = None # circular pitch
P_x = None #axial pitch

power = None # Power in HP
life_time = 15000 # in hours?

T_o = 924



idk how to find Face width, so its going right here

In [1152]:
F = 0.625

In [1153]:
phi_n = 14.5
phi_t = None

Calc p etc etc

In [1154]:
if p is None and P_d is not None:
    p = np.pi / P_d
    P_x = p
    D_g = N_g / P_d

if p is None and P_d is None:
    p = P_x
    P_d = np.pi / P_x
    D_g = N_g / P_d

if p is not None:
    P_x = p
    P_d = np.pi / p
    D_g = N_g / P_d

if D_g is not None:
    P_d = N_g / D_g
    p = np.pi / P_d
    P_x = p


assert(P_d is not None); assert(p is not None); assert(P_x is not None); assert(D_g is not None)
P_d, p, P_x, D_g

(10.0, 0.3141592653589793, 0.3141592653589793, 4.0)

calc L, $\lambda$

In [1155]:
L = N_w * P_x
L

0.3141592653589793

In [1156]:
var_lambda = 180 / np.pi * np.arctan(L / (np.pi * D_w))
var_lambda

4.573921259900861

copmute normal circular pitch

In [1157]:
p_n = p * np.cos(var_lambda  * np.pi / 180)
p_n

0.31315875560351536

Pitch line speed

In [1158]:
##v_t

v_tG = np.pi * D_g * n_g / 12
v_tG

31.415926535897928

Compute Velcoity ratio and gear ratio

In [1159]:

if n_w is not None:
    VR = n_w / n_g
else: 
    VR = N_g / N_w
    n_w = n_g * VR

VR, n_w

(40.0, 1200.0)

In [1160]:
m_g = 1 / VR
m_g

0.025

In [1161]:
assert(VR == N_g / N_w); assert(VR == n_w / n_g)

sliding speed

In [1162]:
v_s = v_tG / np.sin(var_lambda * np.pi / 180)
v_s

393.95371454922224

coefficient of friction

In [1163]:
mu = None
if v_s == 0:
    mu = 0.15
elif v_s < 10:
    mu = 0.124*np.exp(-0.074 * v_s **(0.645) )
elif v_s >= 10:
    mu = 0.103 * np.exp(-0.11 * v_s **(0.45) ) + 0.012

assert(mu is not None)
mu

0.03239664475113534

Compute forces

In [1164]:
assert(T_o is not None)

W_tG = 2 * T_o / D_g
W_tG

462.0

In [1165]:
W_xG = W_tG * ( np.cos(phi_n * np.pi / 180) * np.sin(var_lambda * np.pi / 180) + mu * np.cos(var_lambda * np.pi / 180) ) / ( np.cos(phi_n * np.pi / 180) * np.cos(var_lambda * np.pi / 180) - mu * np.sin(var_lambda * np.pi / 180) ) 
W_xG

52.56038122802861

In [1166]:
W_rG = W_tG * np.sin(phi_n * np.pi / 180) / ( np.cos(var_lambda * np.pi / 180) *np.cos(phi_n * np.pi / 180) - mu * np.sin(var_lambda * np.pi / 180) )
W_rG

120.18478913811488

compute friction force

In [1167]:
W_f = mu * W_tG / ( np.cos(var_lambda * np.pi / 180) *np.cos(phi_n * np.pi / 180) - mu * np.sin(var_lambda * np.pi / 180) )
W_f

15.550698359087072

Power loss due to friction

In [1168]:
P_L = v_s * W_f / 33000
print(f"{P_L=} HP")

P_L=0.18564410249687416 HP


compute output power

In [1169]:
if power is None:
    power = T_o * n_g / 63000
power

0.44

Compute the input power

In [1170]:
##Two different formulas for P_i
P_i = None
if(power is not None):
    P_i = power + P_L
else:
    p_i = -1 * 10**4
    assert(False)
P_i

0.6256441024968742

Compute efficiency $\eta$

In [1171]:
eta = power / P_i
eta

0.7032752298695221

In [1172]:
eta_2 = (np.cos(phi_n) - mu * np.tan(var_lambda)) / (np.cos(phi_n) + mu / np.tan(var_lambda))
eta_2

1.6763027279235652

In [1173]:
# assert(eta - eta_2 < 10**-3)

Find lewis form factor, $y$

In [1174]:

# Create a dictionary from number to number
lewis_form_dict = {14.5: 0.1, 20: 0.125, 25: 0.15, 30: 0.175}

y = lewis_form_dict.get(phi_n)
y



0.1

Compute k_v ???

In [1175]:
k_v = 1200 / (1200 + v_tG)
k_v

0.9744879647413087

compute dynamic load $W_d$

In [1176]:
W_d = W_tG / k_v
W_d

474.0951317163207

Find stress in gear teeth

In [1177]:
sigma = W_d / (y * F * p_n)
sigma

24222.6090496573

Find C_s FROM TABLE???

In [1178]:
C_s = 1000# None
if C_s is None:
    Warning("C_s is None")
# assert(C_s is not None)

FInd C_m

In [1179]:
C_m = None
m_g = VR

if 6 < m_g and m_g < 20:
    C_m = 0.02 * np.sqrt(-1 * m_g**2 + 40*m_g - 76) + 0.46
elif m_g >= 20 and m_g < 76:
    C_m = 0.0107 * np.sqrt(-1 * m_g**2 + 56*m_g + 5146)
elif m_g >= 76:
    C_m = 1.1483 - 0.00658 * m_g

assert(C_m is not None)
C_m

0.8139036429455271

Find C_v

In [1180]:
C_v = None
if 0 < v_s and v_s < 700:
    C_v = 0.659 * np.exp(-0.0011 * v_s)
elif v_s > 700  and v_s < 3000:
    C_v = 13.31 * v_s ** (-0.571)
elif v_s > 3000:
    C_s = 65.52 * v_s ** (-0.774)

assert(C_v is not None)
C_v

0.42725219030387684

Find $F_e$

this formula is possibly wrong

In [1181]:
F_e = None

if F < D_w / 3:
    F_e = F
elif F >= D_w / 3:
    F_e = D_w / 3

assert(F_e is not None)
F_e = F

Find rated tangential load

In [1182]:
W_tR = C_s * (D_g ** 0.8) * F_e * C_m * C_v
W_tR

658.848104103243

Check if the design is satisfactory to resist pitting

In [1183]:
assert(W_tR > W_tG)

Need to find F.

check Mott and Shigley for the formulas in this section. I feel like the pitting formulas are different from the assignment 2.